In [41]:
#import libraries needed
import numpy as np
import random
import hashlib

In [42]:
#define the original coloring for the petersen graph
coloring = {
    # outer five nodes, clockwise from top
    0: 'red',
    1: 'blue', 
    2: 'green',
    3: 'red',
    4: 'blue',
    # inner five nodes, clockwise from top
    5: 'blue',
    6: 'red',
    7: 'red',
    8: 'green',
    9: 'green'
}
print('Number of nodes:', len(coloring))

Number of nodes: 10


In [43]:
#define the set of edges for the graph
#each edge indicates the nodes it attaches and will be used for verification
edges = [
    # outer shape, clockwise from top
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 0),
    # inner shape, clockwise from top
    (5, 0), (5, 7),
    (6, 1), (6, 8),
    (7, 2), (7, 9),
    (8, 3), (8, 5),
    (9, 4), (9, 6)
]
print('Number of edges:', len(edges))

Number of edges: 15


***Non-Interactive, Zero Knowledge Proof Protocol***

**Statement**: Hash Function (sha 256) - used for hiding the colors of the graph and sharing the randomness of choosing a challenge

**Witness**: The Prover's coloring of the Graph


One iteration of the protocol:
- **Commit**: Prover runs their graph coloring algorithm to produce a 3-coloring of the Peterson Graph. The Prover commits to this coloring with the colors represented by their hased equivalents.

- **Challenge**: using a hash function, H (the statement), the user calculates H(m, i) where i is the iteration number and m is commitment the prover generated in the last step. The Prover picks a random edge using the integer representation of this hash to satisfy randomness requirement Fiat Shamir Heuristic and saves this edge as the challenge.

- **Response**: The Prover opens the commitment for the edge in the challenge to the Verifier and submits the colors attaches to this edge.

The proof object consists of:
- the commitment from step 1
- the challenge from step 2 with the iteration number
- the response from step 3

Prover runs the above n times and packages up the n proof objects. These proof objects are then sent to the Verifier.

**Proof Checking**

For each proof object received from the prover the verifier checks
1. the commitment was valid for the views in the response
2. the two views are consistent with each other 
3. The colors of the edge do not match
4. verifier recomputes H(m, i) and regenerates the random edge from the challenge and checks that it matches the edge specified in the prover's challenge. (using the Fiat-Shamir Heuristic)

In [44]:
#declare the container which will hold all of the proof objects from the Prover
#these objects will be verified by the Verifier
proof_objects = []

#the prover knows the witness (coloring) and must prove the knowledge to the Verifier
class Prover:
    
    def shuffle_and_commit(self, iteration):
        #create an array to hold each component of the proof object
        self.proof_object = []
        
        #pick two random color options without replacement
        colors = ['red', 'green', 'blue']
        randColors = random.sample(colors, 2)
        
        #create empty dictionary to commit
        self.local_commitment = {} #will have the raw colors
        commitment = {} #will have the hased colors
        
        #loop through the nodes and assign values
        for i in range(len(coloring)):
            #if the original color is one of the randomly chosen swap the value
            if coloring[i] == randColors[0]: self.local_commitment[i] = randColors[1]
            elif coloring[i] == randColors[1]: self.local_commitment[i] = randColors[0]
            #otherwise keep the same
            else: self.local_commitment[i] = coloring[i]
        
        #hash all of the colorings for the commitment
        #this protects the witness from the Verifier
        for i in range(len(self.local_commitment)):
            hashed_color = hashlib.sha256(bytes(self.local_commitment[i].encode())).hexdigest()
            commitment[i] = hashed_color
        
        #add the hashed commitment to this iteration's proof object
        self.proof_object.append(commitment)
        
        #create a challenge based on this commitment and the iteration number
        self.challenge(commitment, iteration)

    
    def challenge(self, commitment, iteration):
        #convert the commitment (dictionary of coloring) to a string
        string_commit = str(commitment) + str(iteration)
        
        #hash the string to get a unique value (this will determine the edge)
        hashed_commit = hashlib.sha256(bytes(string_commit.encode())).hexdigest()
        
        #convert the hashed value to an int -> use modulo based on the number of edges to pick an edge
        edge_int = int(hashed_commit, 16) % len(edges)
        #choose the edge to check
        edge = edges[edge_int]
        
        #add the challenge to the local proof object with the iteration for the verifier to check
        self.proof_object.append((edge, iteration))
        
        #send the edge to the prover to open the colors
        self.response(edge)
        
    def response(self, edge):
        #save the colors of the nodes connected by the edge from the verifier's commitment
        c1 = self.local_commitment[edge[0]]
        c2 = self.local_commitment[edge[1]]
        
        #add the response to the local proof object
        self.proof_object.append((c1, c2))
        
        #the proof object is complete
        #add the three part proof object to the master list
        proof_objects.append(self.proof_object)

class Verifier:
    
    def check(self):
        #loop through the proof objects for checking
        for p in proof_objects:
            #save the commitment which is the first index
            commitment = p[0]
            #save the challenge (edge) which is the second index
            challenge = p[1]
            #save the response (the two colors) which is the third index
            response = p[2]
            
            #first make sure the two colors don't match
            assert(response[0] != response[1])
            
            #next make sure the colors match the challenge's colors
            #if they don't match there was cheating
            assert(hashlib.sha256(bytes(response[0].encode())).hexdigest() == commitment[challenge[0][0]])
            assert(hashlib.sha256(bytes(response[1].encode())).hexdigest() == commitment[challenge[0][1]])
            
            #last reconstruct which edge was supposed to have been chosen with the shared randomness
            #make sure the edges match
            #convert the commitment (dictionary of coloring) to a string
            string_commit = str(commitment) + str(challenge[1])
        
            #hash the string to get a unique value (this will determine the edge)
            hashed_commit = hashlib.sha256(bytes(string_commit.encode())).hexdigest()
        
            #convert the hashed value to an int -> use modulo based on the number of edges to pick an edge
            edge_int = int(hashed_commit, 16) % len(edges)
            #choose the edge to check
            edge = edges[edge_int]
            
            assert edge == challenge[0]
        

def run_protocol():
    #create instances of both classes
    P = Prover()
    V = Verifier()
    
    #loop through the prover to create 15**2 proof objects for the Verifier
    #picking 15**2 lessens the chance of cheating being missed
    for i in range(15**2):
        P.shuffle_and_commit(i)
    
    #once the proof objects are combiled send them to the verifier
    V.check()
    
run_protocol()

print(proof_objects)

[[{0: 'b1f51a511f1da0cd348b8f8598db32e61cb963e5fc69e2b41485bf99590ed75a', 1: 'ba4788b226aa8dc2e6dc74248bb9f618cfa8c959e0c26c147be48f6839a0b088', 2: '16477688c0e00699c6cfa4497a3612d7e83c532062b64b250fed8908128ed548', 3: 'b1f51a511f1da0cd348b8f8598db32e61cb963e5fc69e2b41485bf99590ed75a', 4: 'ba4788b226aa8dc2e6dc74248bb9f618cfa8c959e0c26c147be48f6839a0b088', 5: 'ba4788b226aa8dc2e6dc74248bb9f618cfa8c959e0c26c147be48f6839a0b088', 6: 'b1f51a511f1da0cd348b8f8598db32e61cb963e5fc69e2b41485bf99590ed75a', 7: 'b1f51a511f1da0cd348b8f8598db32e61cb963e5fc69e2b41485bf99590ed75a', 8: '16477688c0e00699c6cfa4497a3612d7e83c532062b64b250fed8908128ed548', 9: '16477688c0e00699c6cfa4497a3612d7e83c532062b64b250fed8908128ed548'}, ((3, 4), 0), ('red', 'green')], [{0: 'b1f51a511f1da0cd348b8f8598db32e61cb963e5fc69e2b41485bf99590ed75a', 1: 'ba4788b226aa8dc2e6dc74248bb9f618cfa8c959e0c26c147be48f6839a0b088', 2: '16477688c0e00699c6cfa4497a3612d7e83c532062b64b250fed8908128ed548', 3: 'b1f51a511f1da0cd348b8f8598db32e61cb